<a href="https://colab.research.google.com/github/henriqueyjw/pesquisas_insper/blob/main/teste_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pymupdf
import os
import numpy as np
import pandas as pd
import re
from datetime import date
from transformers import T5Tokenizer, T5ForConditionalGeneration

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file and prints the first `num_chars` characters.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = pymupdf.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text


def sumarization(text):
    model_checkpoint = "stjiris/t5-portuguese-legal-summarization"
    t5_model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
    t5_tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
    preprocess_text = text
    t5_prepared_Text = "summarize: "+preprocess_text
    #print ("original text preprocessed: \n", preprocess_text)

    tokenized_text = t5_tokenizer.encode(t5_prepared_Text, return_tensors="pt")

    # summmarize
    summary_ids = t5_model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=512,
                                        max_length=1024,
                                        early_stopping=True)

    output = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output


num_processo = list()
texto = list()
resumo = list()
for arquivo in os.listdir("decisoes_tjsp_hc"):
    if arquivo.endswith(".pdf"):
        num_processo.append(arquivo[:25])
        texto_pdf = extract_text_from_pdf(os.path.join("decisoes_tjsp_hc", arquivo))
        texto.append(texto_pdf)
        resumo.append(sumarization(texto_pdf))

df = pd.DataFrame({"num_processo": num_processo, "texto": texto})
termos_interesse = {r'habeas\scorpus|\shc\s': 'HC'}
split_interesse = list()

for _, row in df.iterrows():

    if re.search(r'habeas\scorpus.{10}|\shc.{10}', row['texto'], re.IGNORECASE|re.MULTILINE):
        match = re.findall(r'habeas\scorpus.{10}|\shc.{10}', row['texto'], re.IGNORECASE|re.MULTILINE|re.DOTALL)
        # split_interesse.append(row['texto'][match.span(0)[0]:match.span(0)[1]+1000])
        split_interesse.append(match)
    else:
        split_interesse.append(None)
df['split_interesse'] = split_interesse

df.to_excel(f"decisoes_tjsp_hc_{date.today()}_split_interesse.xlsx", index=False)